In [1]:
import numpy as np
import cv2
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import SGD, Adam
from keras.models import Sequential
from keras.layers import Convolution2D, ZeroPadding2D, MaxPooling2D
from keras.layers import normalization
import h5py
from test import *
from keras import backend as K
import time
from FaceRec.pretrained_cnn import *                         # Hi Shivam, this is going to take about a second
import FaceRec
from FaceRec.net import *
from FaceRec.get_input import *
from scipy.misc import imsave
import numpy as np


In [17]:
x,y = from_file()                             # This prints some things on the terminal
x_test, y_test = test_file()                  # last statement must be : No. of images 237 no. of classes: 237

In [3]:
model = Sequential()
model.add(Convolution2D(64, 3, 3,input_shape=(3, 227, 227), activation='relu', name='conv1_1', border_mode='same'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(64, 3, 3, activation='relu', name='conv1_2'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(128, 3, 3, activation='relu', name='conv2_1'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(128, 3, 3, activation='relu', name='conv2_2'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_1'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_2'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_3'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_1'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_2'))           # This will take more than a few seconds
model.add(ZeroPadding2D((1, 1)))                                                 # but will finish quickly, just keep a 
model.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_3'))           # watch on the [*]
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_1'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_2'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_3'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

layer_dict = dict([(layer.name, layer) for layer in model.layers])


In [4]:
model.add(Flatten())
model.add(Dense(output_dim=4096, activation='relu', init="uniform"))
model.add(Dense(output_dim=4096, init="uniform", activation='relu'))        # Okey this one takes some time
model.add(Dropout(0.5))                                                     # about 3 minutes and you'll be done
model.add(Dense(output_dim=21, init="uniform", activation='softmax'))       # You can probably start your systems now

In [5]:
f = h5py.File("cnn_weights.h5")
for k in range(f.attrs['nb_layers']):
    if k >= len(model.layers):
        # we don't look at the last (fully-connected) layers in the savefile
        break
    g = f['layer_{}'.format(k)]
    weights = [g['param_{}'.format(p)] for p in range(g.attrs['nb_params'])]     # 2 minutes to this, won't take a lot
    model.layers[k].set_weights(weights)                                          # of time
f.close()

In [18]:
x = x.astype('float32')
x_test = x_test.astype('float32')
x /= 255
x_test /= 255
x = np.subtract(x,np.average(x))
x_test = np.subtract(x_test,np.average(x_test))                    # This is necessary, data preprocessing
y = np_utils.to_categorical(y, 21)                                 # takes about a second
y_test = np_utils.to_categorical(y_test, 21)


In [14]:
sgd = SGD(lr=0.01, decay=5e-4, momentum=0.9, nesterov=True)        # gonna take 1.5-2 minutes
model.compile(loss='categorical_crossentropy', optimizer=sgd)      # I've usually noted 80-90 seconds on this

In [ ]:


######################################################################################################################
##                                                                                                                  ##
##                     You can run this one and get back to your work, this is the training                         ##
##                     of network. It is going to run for about 7-8 hours. If you're gonna                          ##
##                     be in the lab for lesser time, (around 5 hours) change the number in                         ##
##                     xrange to "10", If random timing then each epoch takes about 30 minutes                      ##
##                     calculate wrt your time of stay and put the number accordingly. Thanks.                      ##
##                                                                                                                  ##
######################################################################################################################

for i in xrange(15):                 
    print "\n"
    print "="*30
    print "\n"
    print "Training the epoch ...", i
    model.fit(x, y, nb_epoch=1, batch_size=16, verbose=1,show_accuracy=True, shuffle=True)
    print model.evaluate(x_test, y_test, batch_size=16, show_accuracy=True)
    print model.predict(x_test, batch_size=16)

In [ ]:

######################################################################################################################
##                                                                                                                  ##
##                     Run this once the above snippet stops. This is to save the trained                           ##
##                     weights in order to be able to use them again. If you are going to                           ##
##                     be in the lab for some more time then each epoch takes about 30                              ##
##                     minutes, calculate your time of stay and put the number accordingly in                       ##
##                     xrange snippet above and run it again. This has been the most helpful                        ##
##                     thing for me. I am truly thankful for your cooperation. Thanks                               ##
##                                                                                                                  ##
######################################################################################################################



model.save_weights("cnn_weights.h5",overwrite=True)

In [19]:
y_test

array([[ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  1.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  1.,  0., ...,  0.,  0.,  0.],
       [ 0.,  1.,  0., ...,  0.,  0.,  0.],
       [ 0.,  1.,  0., ...,  0.,  0.,  0.]])

In [21]:
yk = list(y_test)
lst = []
for i in yk:
    i = list(i)
    lst += [i.index(max(i))]
print lst

In [23]:
preds = model.predict(x_test, batch_size=16)
lst = []
for i in preds:
    i = list(i)
    lst += [[i.index(max(i)), max(i)]]
print lst